In [1]:
# load data from mongodb regarding
# commit message, fileactions, lines changed, change type (added, deleted, modified) and author id
import pandas as pd
from bson.objectid import ObjectId
from mongoengine import connect
from pycoshark.mongomodels import Project, VCSSystem, Commit, Tag, File, CodeEntityState, FileAction, People, IssueSystem, Issue, Message, MailingList, Event, MynbouData, Identity, Hunk, Branch
#loc = {'host': '127.0.0.1',
#       'port': 27017,
#       'db': 'smartshark',
#       'username': 'root',
#       'password': '',
#       'authentication_source': 'smartshark',
#       'connect': False}
#connect(**loc)
bimbes = {'host': '94.130.131.235',
       'port': 27017,
       'db': 'smartshark',
       'username': 'atrautsch',
       'password': 'balla',
       'authentication_source': 'smartshark',
       'connect': False}
connect(**bimbes)


MongoClient(host=['94.130.131.235:27017'], document_class=dict, tz_aware=False, connect=False, read_preference=Primary())

In [2]:
dfall = pd.read_pickle('../data/aggregated_full.pickle')
len(dfall)

112267

In [3]:
dfall['commit_msg'] = ''
dfall['files_added'] = 0
dfall['files_deleted'] = 0
dfall['files_modified'] = 0

for project_name in dfall['project'].unique():
    pr = Project.objects.get(name=project_name)
    vcs = VCSSystem.objects.get(project_id=pr.id)
    
    for revision_hash in dfall[dfall['project'] == project_name]['revision'].unique():
        # fetch commit message
        files_added = 0
        files_deleted = 0
        files_modified = 0
        c = Commit.objects.only('id', 'author_id', 'message').get(vcs_system_id=vcs.id, revision_hash=revision_hash)
        
        for fa in FileAction.objects.filter(commit_id=c.id):
            if fa.mode == 'M':
                files_modified += 1
            elif fa.mode in ['A', 'C']:
                files_added += 1
            elif fa.mode == 'D':
                files_deleted += 1
        
        f = dfall[(dfall['project'] == project_name) & (dfall['revision'] == revision_hash)]
        dfall.loc[f.index, 'author_id'] = c.author_id
        dfall.loc[f.index, 'commit_msg'] = c.message
        dfall.loc[f.index, 'files_added'] = files_added
        dfall.loc[f.index, 'files_modified'] = files_modified
        dfall.loc[f.index, 'files_deleted'] = files_deleted
        

In [4]:
dfall.to_pickle('../data/aggregated_full_enriched2.pickle')

In [5]:
dfall.head()

,revision,date,code_PMD_ABSALIL,code_PMD_ADLIBDC,code_PMD_AMUO,code_PMD_ATG,code_PMD_AUHCIP,code_PMD_AUOV,code_PMD_BII,code_PMD_BI,...,major_only_time_ratio,critical_only_time,critical_only_time_ratio,year,use_maven,commit_msg,files_added,files_deleted,files_modified,author_id
0,02020af0a933e27dc6351e5a6a136a527c4be305,2006-01-01 02:55:24,0,0,0,0,0,0,0,0,...,3.301692,8,3.301692,2006,1,add digester in a utility project\nPR: MRM-9\n...,3,0,0,58bfd7cb3cff546cc7b44e19
1,50e313727bb5f425bb74c53685ac5a36ef2e8f67,2006-01-01 02:55:45,0,0,0,0,0,0,0,0,...,4.003203,11,4.403523,2006,1,add utils project\n\n\ngit-svn-id: https://svn...,0,0,1,58bfd7cb3cff546cc7b44e19
2,50c62c3db718315010f0f07b6a2d4c27b28b88a1,2006-01-01 02:56:07,0,0,0,0,0,0,0,0,...,4.003203,11,4.403523,2006,1,remove outdated comments\n\n\ngit-svn-id: http...,0,0,1,58bfd7cb3cff546cc7b44e19
3,9a8157875a77cad42c53433426ffd65a4864a304,2006-01-01 02:57:59,0,0,0,0,0,0,0,0,...,3.794266,9,3.794266,2006,1,"- improve cache, make it more map like and syn...",0,0,12,58bfd7cb3cff546cc7b44e19
4,1c39abca5704dfd68f69bbb97375a8d948a22249,2006-01-01 02:58:35,0,0,0,0,0,0,0,0,...,3.006873,8,3.436426,2006,1,use new digester and clean up related code\n\n...,0,0,2,58bfd7cb3cff546cc7b44e19


In [ ]:
# count complete commits our study subjects have
commits = 0
for project_name in dfall['project'].unique():
    p = Project.objects.get(name=project_name)
    vcs = VCSSystem.objects.get(project_id=p.id)
    
    commits += Commit.objects.filter(vcs_system_id=vcs.id).count()

print('study subjects have', commits, 'number of commits')